DAMI 2 project maan5745

In [ ]:
#Import libraries

import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Load data

training_data = pd.read_csv("datasets/aps_failure_training_set.csv",skiprows=20, na_values='na')
test_data = pd.read_csv("datasets/aps_failure_test_set.csv",skiprows=20, na_values='na')



In [ ]:
print(training_data.shape)
training_data.head()

In [ ]:
print(test_data.shape)
test_data.head()

In [ ]:
# zeros in the training set
zeros_train = (training_data == 0).sum()/len(training_data)
plt.figure(figsize=(25,5))
zeros_train.plot.bar()

In [ ]:
# zeros in the test set
zeros_test = (test_data == 0).sum()/len(test_data)
plt.figure(figsize=(25,5))
zeros_test.plot.bar()

In [ ]:
nan_training = (training_data.isna().sum())/len(training_data)
plt.figure(figsize=(25,5))
nan_training.plot.bar()


In [ ]:
sns.barplot(training_data['class'].unique(),training_data['class'].value_counts())
plt.title('Distribution of Class Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()
print('count of ones:',training_data['class'].value_counts()[1])
print('count of zeros: ',training_data['class'].value_counts()[0])
print('ratio ones/zeros: ',training_data['class'].value_counts()[1]/training_data['class'].value_counts()[0])

In [ ]:
sns.barplot(test_data['class'].unique(),test_data['class'].value_counts())
plt.title('Distribution of Class Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()
print('count of ones: ',test_data['class'].value_counts()[1])
print('count of zeros: ',test_data['class'].value_counts()[0])
print('ratio ones/zeros: ',test_data['class'].value_counts()[1]/test_data['class'].value_counts()[0])

In [ ]:

std_training = training_data.std()
std_training.sort_values(ascending=True)

In [ ]:
training_data['cd_000'].plot()

In [ ]:
# drop feature cd_000 due to constant values
training_data.drop('cd_000',axis =1, inplace=True)

In [ ]:
# Drop NaN according to threshold = 0,5

threshold = len(training_data)*0.5

training_data_dropped = training_data.dropna(axis=1, thresh=threshold)


col_diff = test_data.columns.difference(training_data_dropped.columns)
test_data_dropped = test_data.drop(col_diff, axis =1)
training_data_dropped.info(), test_data_dropped.info()

In [ ]:
training_data_dropped.columns[training_data_dropped.isna().any()].to_list


In [ ]:
# Imputing remaining NaN values

training_data_dropped_imputed = training_data_dropped.copy()
training_data_dropped_imputed.drop('class', axis =1, inplace=True)
columns = list(training_data_dropped.columns)
columns.pop(0)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(training_data_dropped_imputed)
training_data_dropped_imputed = imputer.transform(training_data_dropped_imputed)
training_data_dropped_imputed =pd.DataFrame(training_data_dropped_imputed, columns=columns)

training_data_dropped_imputed.columns[training_data_dropped_imputed.isna().any()].to_list

In [ ]:
test_data_dropped_imputed = test_data_dropped.copy()
test_data_dropped_imputed.drop('class', axis =1, inplace=True)

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(test_data_dropped_imputed)
test_data_dropped_imputed = imputer.transform(test_data_dropped_imputed)
test_data_dropped_imputed =pd.DataFrame(test_data_dropped_imputed, columns=columns)

test_data_dropped_imputed.columns[test_data_dropped_imputed.isna().any()].to_list

In [ ]:
# Setting up the final version of the data set for the classification 

X_train = training_data_dropped_imputed.copy()
y_train = training_data_dropped['class']
y_train.replace({'neg': 0, 'pos': 1}, inplace=True)

X_test = test_data_dropped_imputed.copy()
y_test = test_data_dropped['class']
y_test.replace({'neg': 0, 'pos': 1}, inplace=True)

In [ ]:
# Random forest within the genetic algo using the Scania cost function for evaluation

from geneticalgorithm import geneticalgorithm as ga

def scania_cost_func(X):

    #n_est = int(n_est)
    #maxdepth = int(maxdepth)

    rf = RandomForestClassifier(n_estimators = int(X[0]), max_depth = X[1], random_state = 123)
    rf.fit(X_train, y_train)
        
    true = y_test
    pred = rf.predict(X_test)

    a = 0
    b = 0
    c = 0
    d = 0

    for i in range(len(true)):
        if ((true[i] == 1) & (pred[i] == 1)):
            a = a + 1
        elif ((true[i] == 1) & (pred[i] == 0)):
            b = b + 1
        elif ((true[i] == 0) & (pred[i] == 1)):
            c = c + 1
        elif ((true[i] == 0) & (pred[i] == 0)):
            d = d + 1
    
    cost = c*10+b*500

    return cost
algorithm_param = {'max_num_iteration': 100,\
                   'population_size':100,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':0.5,\
                   'max_iteration_without_improv':None}

varbound=np.array([[2,30],[1,10]])
#vartype=np.array([['int'],['int']])
model=ga(function=scania_cost_func,dimension=2,variable_type='int',variable_boundaries=varbound,function_timeout=40)

model.run()
convergence=model.report


In [ ]:
# printing the confusion matrix for the best hyperparamters

rf = RandomForestClassifier(n_estimators = 19, max_depth = 10, random_state = 123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
confusion_matrix(y_test, y_pred)